# 3.1深度森林介绍
深度森林(Deep Forest)是周志华教授和冯霁博士在2017年2月28日发表的论文《Deep Forest: Towards An Alternative to Deep Neural Networks》中提出来的一种新的可以与深度神经网络相媲美的基于树的模型，可以在除了⼤规模图像数据之外的任务中获得和深度神经网络相似的表现。模型有两个关键性质：级联树结构以及多粒度，其结构如图所示：

![](https://ai-studio-static-online.cdn.bcebos.com/b68466fa0fbb436fb2fa7b13ab8ac3a449526b4e95794cd399543bb1ff645f1b)

红⾊标出了原始的输⼊特征向量，基于输⼊，可以训练出⼀些森林，每个森林都是多个决策树的集成模型。假设有三个分类需要预测，每个分类⽤⼀个bit表示，那么每个森林的输出就有3个bit⻓。在第⼀个标签训练完成后，把这3bit输出和原始的输⼊特征向量串联起来， ⽤更多的特征扩增了原始的输⼊向量；接着，⽤扩增后的向量，训练模型的下⼀层。不断重复这个过程，直到最后⼀个标签，对所有预测值取平均，得到最终的预测结果。模型的最终层数可以由各种指标决定，⽐如当发现增加更多的层之后模型的表现并没有提升，这时候就可以停下来。这样就不需要在开始训练前就设定好模型有多少层。这是深度森林非常方便的一点。    
<br/>
<br/>
<br/>

级联树即每一层都由多个随机森林组成。通过随机森林学习输入特征向量的特征信息，经过处理后输入到下一层。为了增强模型的泛化能力，每一层选取多种不同类型的随机森林，上图给了两种随机森林结构，分别为completely-random tree forests（蓝色）和random forests（黑色），每种两个。其中，每个completely-random tree forests包含1000棵树，每个节点通过随机选取一个特征作为判别条件，并根据这个判别条件生成子节点，直到每个叶子节点只包含同一类的实例而停止；每个random forests同样包含1000棵树，节点特征的选择通过随机选择√d个特征（d为输入特征的数量），然后选择基尼系数最大特征作为该节点划分的条件。
每个森林中都包括好多棵决策树，每个决策树都会决策出一个类向量结果（以3类为例，下面也是），然后综合所有的决策树结果，再取均值，生成每个森林的最终决策结果——一个3维类向量！每个森林的决策过程如下图所示。

![](https://ai-studio-static-online.cdn.bcebos.com/3487c2e93d244d369fff6c5e904c3ad8cb3a4199be4d4e969cf393f92e05b42b)

每个森林都会决策出一个3维类向量，回到图1中，级联森林中的4个森林就都可以决策出一个3维类向量，然后对4个*3维类向量取均值，最后取最大值对应的类别，作为最后的预测结果！    
<br/>
<br/>
<br/>

多粒度扫描是为了增强级联森林，为了对特征做更多的处理的一种技术手段,具体扫描过程如下图所示

![](https://ai-studio-static-online.cdn.bcebos.com/2816ac8869b441c6969793fde056043d36c878c3264d491ebfb1f5b7e674a20f)

对于400维的序列数据，采用100维的滑动窗对输入特征进行处理，得到301（400 - 100 + 1）个100维的特征向量。对于20×20的图像数据，采用10×10的滑动窗对输入特征进行处理，得到121（(20-10+1)*(20-10+1)）个10×10的二维特征图。然后将得到的特征向量（或特征图）分别输入到一个completely-random tree forest和一个random forest中（不唯一，也可使用多个森林），以三分类为例，会得到301（或121）个3维类分布向量，将这些向量进行拼接，得到1806（或726）维的特征向量。

给定数据以后，⾸先会做⼀些扫描，⽤窗⼝扫描⼀些样本以后得到特征，并得到数据的表征。通过使⽤不同⼤⼩的窗⼝，也就是不同的粒度（grain）；对于每⼀种粒度，都有许多集成模型。所以全局来看，这就是⼀群级联模型的级联模型；每个级联模型⾥有多种不同的粒度，每个粒度⾥含有集成模型的集成模型。模型的结构也许复杂，但是它可以做得很深。这种模型也会涉及到⼀些超参数问题，⽐如，每个森林⾥有多少树、树生长到什么深度或者什么时间、扫描数据的时候要使⽤多少个不同的窗⼝，但总的来说涉及到的超参数的数量还是⽐神经⽹络少。
<br/>
<br/>
<br/>

深度森林相比于神经网络存在诸多优势。从本质上讲，gcForest是一种新颖的决策树集合方法，其预测准确性与广泛任务中的深度神经网络具有高度竞争性。此外，gcForest更容易训练，因为它具有较少的超参数。而且实验表明，通过使用几乎相同的超参数设置，gcForest可以在不同域上的数据集上实现高预测准确性。另一个优点是可以自动为不同的训练数据集确定gcForest的模型复杂度，使gcForest即使在小规模数据集上也能很好地运行。


# 3.4与随机森林比较
第一问：是真正优于的

<br/>

深度森林每一层都由多个随机森林组成。通过随机森林学习输入特征向量的特征信息，经过处理后输入到下一层。为了增强模型的泛化能力，每一层选取多种不同类型的随机森林。
每个森林中都包括好多棵决策树，每个决策树都会决策出一个类向量结果（以3类为例，下面也是），然后综合所有的决策树结果，再取均值，生成每个森林的最终决策结果——一个3维类向量，这样，每个森林都会决策出一个3维类向量。为降低过拟合风险，每个森林产生的类向量由k折交叉验证产生，即每个样本会被训练k-1次，产生k-1个类向量，然后对其求平均值，以产生作为级联中下一级的增强特征的最终类向量。需要注意的是，在扩展一个新的级后，整个级联的性能将在验证集上进行评估，如果没有显著的性能增益，则训练过程终止。因此，级联中级的数量是自动确定的。与模型的复杂度固定的大多数神经网络相反，深度森林能够适当的终止训练来决定其模型的复杂度。所以深度森林适用于不同规模的训练数据。
<br/>
<br/>
<br/>

![](https://ai-studio-static-online.cdn.bcebos.com/9aaf692c9d7d4402a72dd87e93187f8bf119fbeb4f234a05b9a19091b1c6edfe)

<br/>
<br/>
<br/>

第二问：在大数据集，不同规模的数据集下会优于随机森林，在单数据集下会劣于随机森林

<br/>

![](https://ai-studio-static-online.cdn.bcebos.com/1493c9cede0d4ba795c7900ee1b914b5c0d96540c7864c43b7a7501993d1ab76)

相比随机森林，深度森林有如下若干优点：<br/>
1.容易训练，计算开销小在几乎完全一样的超参数设置下，在处理不同领域的不同数据时，也能达到极佳的性能。<br/>
2.天然适用于并行的部署，效率高<br/>
3.超参数少，模型对超参数调节不敏感，并且一套超参数可使用到不同数据集<br/>
4.可以适应于不同大小的数据集，模型复杂度可自适应伸缩<br/>
5.每个级联的生成使用了交叉验证，避免过拟合<br/>
6.在理论分析方面也比随机森林更加容易。<br/>




请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 